<a href="https://colab.research.google.com/github/arunm917/Climate-Action-Tool/blob/main/equation_solver_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading packages and files

In [1]:
import numpy as np
import pandas as pd
import sympy as sp
import gdown
import sympy
import regex as re

In [2]:
# from google.colab import files
# uploaded = files.upload()

In [3]:
# downloading file from gdrive
output = 'equations'
file_id = '1qEMQT4lnXe0X2WgNZ2QHciH1lw_rDsgp' # Google drive ID
#Download the file
gdown.download('https://drive.google.com/uc?id=' + file_id, output, quiet=False)
print('\nDONE.')

Downloading...
From: https://drive.google.com/uc?id=1qEMQT4lnXe0X2WgNZ2QHciH1lw_rDsgp
To: /content/equations
100%|██████████| 80.0/80.0 [00:00<00:00, 37.9kB/s]


DONE.


In [4]:
# downloading file from gdrive
output = 'parameters'
file_id = '1qEVz2N1atZjRtPK4kGYbZp-bOIrctm2V' # Google drive ID
#Download the file
gdown.download('https://drive.google.com/uc?id=' + file_id, output, quiet=False)
print('\nDONE.')

Downloading...
From: https://drive.google.com/uc?id=1qEVz2N1atZjRtPK4kGYbZp-bOIrctm2V
To: /content/parameters
100%|██████████| 43.0/43.0 [00:00<00:00, 100kB/s]


DONE.


In [5]:
# downloading file from gdrive
output = 'variables'
file_id = '1qFxa_f1ApVT0BBi_ed826kZ9ombKqap5' # Google drive ID
#Download the file
gdown.download('https://drive.google.com/uc?id=' + file_id, output, quiet=False)
print('\nDONE.')

Downloading...
From: https://drive.google.com/uc?id=1qFxa_f1ApVT0BBi_ed826kZ9ombKqap5
To: /content/variables
100%|██████████| 33.0/33.0 [00:00<00:00, 110kB/s]


DONE.


# Processing files

In [6]:
parameters = pd.read_csv('parameters')

In [7]:
parameters

,parameters,values
0,phi1,3
1,phi2,2
2,phi3,2


In [8]:
variables_df = pd.read_csv('variables')
variables_df

,variables,values
0,x,0
1,y,0
2,z,0


In [9]:
variables_list = list(variables_df['variables'])

In [10]:
print(variables_list)

['x', 'y', 'z']


In [11]:
with open('equations', 'r') as f:
    # Read the equations line by line
    eq_lines = f.readlines()

In [12]:
print(eq_lines)

['eq1: x + y - phi1 * z - 10\n', 'eq2: x - y + phi2 * z - 3\n', 'eq3: phi3 * x + y - z + 6']


In [13]:
# Create a dictionary to store the equations
eq_list = []

# Loop through the equation lines
for eq_line in eq_lines:
    # Split the line into the equation name and the equation expression
    eq_name, eq_expr = eq_line.strip().split(':')
    # Convert the tuple of symbols to a single expression
    eq_list.append(eq_expr) 
    # Add the equation to the dictionary

In [14]:
# Creating dictionary for parameters and values
param_dict = dict(zip(parameters['parameters'],parameters['values']))
print(param_dict)

{'phi1': 3, 'phi2': 2, 'phi3': 2}


In [15]:
# Substituting parameters in equation with their values
modified_list = []
for key in param_dict:
    print(key)
    for eq in eq_list:
        if key in eq:
            eq = eq.replace(key, str(param_dict.get(key)))
            modified_list.append(eq)
print(modified_list)

phi1
phi2
phi3
[' x + y - 3 * z - 10', ' x - y + 2 * z - 3', ' 2 * x + y - z + 6']


In [16]:
for item in modified_list:
  print(item)

 x + y - 3 * z - 10
 x - y + 2 * z - 3
 2 * x + y - z + 6


#Extracting coefficients to setup optimization problem

In [45]:
equations_list = modified_list
# variables_list
coefficients_nested_list = []
constants_list = []

for equation in equations_list:
  coefficients_list = []
  # print('Equation:', equation)
  equation_split = equation.split()
  # print('Equation_split:', equation_split)
  i = 1
  for variable in variables_list:
    coeff = 0
    # print('i:',i)
    # print('variable:', variable)
    if variable in equation_split:
      variable_index = equation_split.index(variable)
      # print('variable_index:', variable_index)
      if i == 1:
        if variable_index == 0:
          coeff = 1
          coefficients_list.append(float(coeff))
        
        if variable_index == 1:
          coeff = -1
          coefficients_list.append(float(coeff))
        
        if variable_index == 2:
          coeff = equation_split[variable_index - 2]
          coefficients_list.append(float(coeff))

      else:
        if equation_split[variable_index - 1] == '-' or '+':
          # print(equation_split[variable_index - 1])
          sign = equation_split[variable_index - 1]
          if sign == '+':
            coeff = +1
            coefficients_list.append(float(coeff))
          if sign == '-':
            coeff = -1
            coefficients_list.append(float(coeff))

        if equation_split[variable_index - 1] == '*':
          # print(equation_split[variable_index - 1])
          sign = equation_split[variable_index - 3]
          number = equation_split[variable_index - 2]
          coeff_list = [sign, number]
          # print(coeff_list)
          coeff = float(''.join(coeff_list))
          coefficients_list.append(coeff)
          
      i += 1
    else:
      coeff = 0
      coefficients_list.append(float(coeff))

    
  coefficients_nested_list.append(coefficients_list)
  constant = [equation_split[-2], equation_split[-1]]
  constant = float(''.join(constant))
  constants_list.append(-1*constant)
  # print(constants_list)
  # print(coefficients_list)

print('A:', coefficients_nested_list)
print('b:', constants_list)

A: [[1.0, 1.0, -3.0], [1.0, -1.0, 2.0], [2.0, 1.0, -1.0]]
b: [10.0, 3.0, -6.0]


# Solving the optimization problem

In [46]:
import numpy as np
from scipy.optimize import minimize

In [47]:
# Define the matrix A and the vector b
# A = np.array([[1, 1, -3], [1, -1, 2], [2, 1, -1]])
# b = np.array([10, 3, -6])
A = np.array(coefficients_nested_list)
b = np.array(constants_list)

In [48]:
# Define the objective function
def objective(x):
    print(np.linalg.norm(A.dot(x) - b, 2))
    return np.linalg.norm(A.dot(x) - b, 2)

In [49]:
# Define the constraint
def constraint(x):
    return A.dot(x) - b

In [50]:
# Define the initial guess for x
x0 = np.array([1, 1, 1])


In [51]:
# Define the optimization problem
problem = {'type': 'eq', 'fun': constraint}


In [52]:
# Solve the optimization problem
solution = minimize(objective, x0, constraints=problem)

# Print the solution
print("Solution: ", solution.x)

13.638181696985855
13.638181701356281
13.638181694800643
13.638181722115798
7.944109290391274e-15
3.650024730144612e-08
2.580956622835879e-08
5.575504270097779e-08
Solution:  [  2. -19.  -9.]
